### Dissertação de Mestrado: Frederico Gomes

• **Autor:** Maria Luiza Campos

• **Data:** Julho 2025

**Metodologia:** Teste Exato de Fisher.

In [7]:
## Importando bibliotecas

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Importando os dados

dados = pd.read_csv('dados_limpos.csv')
print("Dados importados com sucesso!")

Dados importados com sucesso!


# Entendendo o modelo de análise

As variáveis de idade, da forma como estão agrupadas, não comportam um teste exato de Fisher, que necessita de duas categorias não ordinais. 
Mesmo que agrupassemos as idades, ainda não seria ideal porque idade possui ordem cronológica (é uma variável quantitativa ordinal). Assim, nossas categorias possuem
uma ordem de compreensão, sendo 0 indivíduos mais novos e 3 indivíduos mais velhos (em ambos réu e vítima).

As demais variáveis são qualitativas nominais, ou seja, as categorias não possuem uma ordem pré-estabelecida e são todas binárias (0,1). Assim, são variáveis ideais para o teste exato de Fisher. Vamos começar nossa análise com essas demais variáveis.

### Teste exato de Fisher

Esse teste vai nos retornar a existência ou não de uma associação entre as variáveis. Ou seja, não retorna uma correlação - que possui direção e força. O p-valor vai apenas nos informar se existe ou não uma associação válida entre as duas variáveis qualitativas nominais binárias. 

H0: não existe associação entre as variáveis. 

Se p-valor for pequeno, então rejeitamos a hipótese nula e podemos afirmar que há associação entre as variáveis. 

O odds ratio do teste nos fornece informação sobre força e direção da relação. 

### Notas relevantes sobre a amostra 

O número de observações se altera a depender da variável independente analisada. Veja abaixo um modelo simples e genérico de equação que representa, visualmente, uma das associações que estamos investigando:

Resultado da Condenação [f(x) = y] = B0 + B1 * Existência de filhos (x)

Aqui, estamos apenas supondo que o resultado da condenação é uma função da existência de filhos, isto é, a existência de filhos (x, variável independente) afeta o resultado da condenação (y, variável dependente). 

As variáveis independentes na amostra tem um número variável de observações, ou seja, o tamanho da amostra muda a depender de x. Nesse caso, eu vou trabalhar com sub-amostras pra cada x numa tentativa de exaurir completamente a informação disponível e tirar vantagem de possíveis amostras maiores. 




In [8]:
### Analisando a categoria de vínculo

## Filtrando apenas para as variáveis de interesse

variaveis_interesse = ['ID_processo','resultado_julgamento', 'categoria_vinculo']
dados_filtrados = dados[variaveis_interesse].copy()
dados_filtrados = dados_filtrados[dados_filtrados['ID_processo'] != 202146] ## removendo um dos processos
print(dados_filtrados)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados)}")

     ID_processo  resultado_julgamento  categoria_vinculo
0         148088                     1                1.0
1         203354                     1                0.0
2         211280                     1                1.0
3         264516                     1                1.0
4         849912                     1                1.0
..           ...                   ...                ...
126       197716                     0                1.0
127       152449                     0                1.0
128       772844                     0                1.0
130       210396                     0                NaN
131       191972                     0                NaN

[131 rows x 3 columns]
Total de observações originais: 132
Total de observações final: 131


In [ ]:
## Verificando missing values

for var in variaveis_interesse:
    missing_count = dados_filtrados[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados[var].unique()}")


ID_processo:
  Missing values: 0 (0.0%)
  Valores únicos: [148088 203354 211280 264516 849912 804741 254352 202422 224010 200221
 184262 132092 835236 173028 191800 193481 694900 184789 174559 146576
 143912 280782 212700 215458 885416 913497 208333 212840 871265 249385
 157443 239068 259005 176898 176979 527774 171531 176559 169452 149635
 175618 140372 143672 155742 158776 231453 134780 195164 831541 800685
 795482 210952 208602 722014 681080 676155 199917 197973 197199 208631
 237957 147619 183064 201216 195723 169551 208489 226153 185259 630372
 188501 189990 142704 163831 153688 379763 477088 382196 387667 160739
 185437 163595 176254 171010 136005 165091 155347 158511 162216 154015
 728575 110419 283470 966382 170792 216534 961744 159217 270098 211850
 214764 231698 248059 279235 166443 172188 118368 221858 230572 201531
 265254 240573 238961 201966 202969 206484 204528 202900 155503 897015
 217780 211854 210161 197716 152449 772844 210396 191972]
resultado_julgamento:
  Missing 

In [10]:
## Vamos remover esses valores faltantes da tabela filtrada

dados_limpos = dados_filtrados.dropna()

for var in variaveis_interesse:
    observacoes_variavel = len(dados_limpos[var])
    print(f"{var}: {observacoes_variavel} observações")

## Isso significa que temos apenas 107 observações com informação COMPLETA. 

ID_processo: 107 observações
resultado_julgamento: 107 observações
categoria_vinculo: 107 observações


In [12]:
## Vamos rodar o Teste Exato de Fisher para cada uma das variáveis de interesse

## Definindo uma função para o teste

def teste_fisher(var1, var2, dados):
    """
    Executa o teste exato de Fisher entre duas variáveis de interesse

    """
    # Criar tabela de contigência
    tabela = pd.crosstab(dados[var1], dados[var2])

    print(f"\n=== Teste de Fisher: {var1} vs {var2} ===")
    print("Tabela de contigência")
    print(tabela)

    # verificar se a tabela é válida para Fisher
    if tabela.shape != (2, 2):
        print(f"⚠️  Tabela {tabela.shape} - Fisher pode não ser apropriado")
        return None, None, None

    # Executar teste de fisher
    try:
        odds_ratio, p_value = stats.fisher_exact(tabela)

        print(f"Odds Ratio: {odds_ratio:.4f}")
        print(f"P-value: {p_value:.6f}")

         # Interpretação
        if p_value < 0.001:
            sig = "*** (p < 0.001)"
        elif p_value < 0.01:
            sig = "** (p < 0.01)"
        elif p_value < 0.05:
            sig = "* (p < 0.05)"
        else:
            sig = "ns (p >= 0.05)"
        
        print(f"Significância: {sig}")
        
        return odds_ratio, p_value, sig
        
    except Exception as e:
        print(f"❌ Erro no teste: {e}")
        return None, None, None


In [13]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 1: resultado_julgamento vs categoria_vinculo

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'categoria_vinculo', dados_limpos)
resultados['resultado_julgamento_vs_categoria_vinculo'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs categoria_vinculo ===
Tabela de contigência
categoria_vinculo     0.0  1.0
resultado_julgamento          
0                       0   13
1                      46   48
Odds Ratio: 0.0000
P-value: 0.000499
Significância: *** (p < 0.001)

resultado_julgamento_vs_categoria_vinculo:
  Odds Ratio: 0.0000
  P-valor: 0.000499
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Categoria de Vínculo**

• Odds Ratio: 0.

• P-valor: 0.000410

1. Odds ratio = 0: Uma das células da tabela de contingência é zero. Nenhum caso do tipo abusivo na amostra foi absolvido. 

2.  Dos casos com vínculo afetivo na amostra, 48 foram condenados e 13 absolvidos. 

3.  Dos casos considerados como abuso na amostra, 46 foram condenados. 

**Conclusão:** *Ter vínculo afetivo pode alterar a decisão do julgamento*.

**• O p-valor está abaixo de 0.05 (0.000499), indicando uma associação forte entre resultado do julgamento e tipo de vínculo.**

In [14]:
## Analisando Coabitação

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_2 = ['ID_processo','resultado_julgamento', 'coab']
dados_filtrados_2 = dados[variaveis_interesse_2].copy()
dados_filtrados_2 = dados_filtrados_2[dados_filtrados_2['ID_processo'] != 202146] ## removendo um dos processos
print(dados_filtrados_2)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_2)}")

     ID_processo  resultado_julgamento  coab
0         148088                     1   NaN
1         203354                     1   0.0
2         211280                     1   NaN
3         264516                     1   NaN
4         849912                     1   0.0
..           ...                   ...   ...
126       197716                     0   1.0
127       152449                     0   NaN
128       772844                     0   NaN
130       210396                     0   1.0
131       191972                     0   NaN

[131 rows x 3 columns]
Total de observações originais: 132
Total de observações final: 131


In [ ]:
## Verificando missing values

for var in variaveis_interesse_2:
    missing_count = dados_filtrados_2[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_2)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_2[var].unique()}")

## Das 131 observações, 76 são missing values. Talvez essa variável não seja relevante mais. 

ID_processo:
  Missing values: 0 (0.0%)
  Valores únicos: [148088 203354 211280 264516 849912 804741 254352 202422 224010 200221
 184262 132092 835236 173028 191800 193481 694900 184789 174559 146576
 143912 280782 212700 215458 885416 913497 208333 212840 871265 249385
 157443 239068 259005 176898 176979 527774 171531 176559 169452 149635
 175618 140372 143672 155742 158776 231453 134780 195164 831541 800685
 795482 210952 208602 722014 681080 676155 199917 197973 197199 208631
 237957 147619 183064 201216 195723 169551 208489 226153 185259 630372
 188501 189990 142704 163831 153688 379763 477088 382196 387667 160739
 185437 163595 176254 171010 136005 165091 155347 158511 162216 154015
 728575 110419 283470 966382 170792 216534 961744 159217 270098 211850
 214764 231698 248059 279235 166443 172188 118368 221858 230572 201531
 265254 240573 238961 201966 202969 206484 204528 202900 155503 897015
 217780 211854 210161 197716 152449 772844 210396 191972]
resultado_julgamento:
  Missing 

In [ ]:
## Vamos remover esses valores faltantes da tabela filtrada

dados_limpos_2 = dados_filtrados_2.dropna()

for var in variaveis_interesse_2:
    observacoes_variavel = len(dados_limpos_2[var])
    print(f"{var}: {observacoes_variavel} observações")

## Isso significa que temos apenas 55 observações com informação COMPLETA. 

ID_processo: 55 observações
resultado_julgamento: 55 observações
coab: 55 observações


In [17]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 2: resultado_julgamento vs coabitação

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'coab', dados_limpos_2)
resultados['resultado_julgamento_vs_coabitacao'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs coab ===
Tabela de contigência
coab                  0.0  1.0
resultado_julgamento          
0                       0   10
1                      35   10
Odds Ratio: 0.0000
P-value: 0.000006
Significância: *** (p < 0.001)

resultado_julgamento_vs_coabitacao:
  Odds Ratio: 0.0000
  P-valor: 0.000006
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Coabitação**

• Odds Ratio: 0.

• P-valor: 0.000006

1. Odds ratio = 0: Uma das células da tabela de contingência é zero. Nenhum caso do tipo sem coabitação foi absolvido. 

2.  Dos casos com coabitação, 10 foram condenados e 10 absolvidos. 

3.  Dos casos sem coabitação, 35 foram condenados. 

**Conclusão:** *a coabitação altera a decisão do julgamento*.

**• O p-valor está abaixo de 0.05 (0.000006), indicando uma associação forte entre resultado do julgamento e coabitação.**

**• Esse resultado deve ser analisado com cautela porque essa amostra é pequena.**


In [18]:
## Analisando existência de filhos

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_3 = ['ID_processo','resultado_julgamento', 'filhos']
dados_filtrados_3 = dados[variaveis_interesse_3].copy()
dados_filtrados_3 = dados_filtrados_3[dados_filtrados_3['ID_processo'] != 202146] ## removendo um dos processos
print(dados_filtrados_3)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_3)}")

     ID_processo  resultado_julgamento  filhos
0         148088                     1     0.0
1         203354                     1     0.0
2         211280                     1     0.0
3         264516                     1     NaN
4         849912                     1     1.0
..           ...                   ...     ...
126       197716                     0     1.0
127       152449                     0     1.0
128       772844                     0     0.0
130       210396                     0     0.0
131       191972                     0     1.0

[131 rows x 3 columns]
Total de observações originais: 132
Total de observações final: 131


In [ ]:
## Verificando missing values

for var in variaveis_interesse_3:
    missing_count = dados_filtrados_3[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_3)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_3[var].unique()}")

# 32% da amostra tem missing values. 

ID_processo:
  Missing values: 0 (0.0%)
  Valores únicos: [148088 203354 211280 264516 849912 804741 254352 202422 224010 200221
 184262 132092 835236 173028 191800 193481 694900 184789 174559 146576
 143912 280782 212700 215458 885416 913497 208333 212840 871265 249385
 157443 239068 259005 176898 176979 527774 171531 176559 169452 149635
 175618 140372 143672 155742 158776 231453 134780 195164 831541 800685
 795482 210952 208602 722014 681080 676155 199917 197973 197199 208631
 237957 147619 183064 201216 195723 169551 208489 226153 185259 630372
 188501 189990 142704 163831 153688 379763 477088 382196 387667 160739
 185437 163595 176254 171010 136005 165091 155347 158511 162216 154015
 728575 110419 283470 966382 170792 216534 961744 159217 270098 211850
 214764 231698 248059 279235 166443 172188 118368 221858 230572 201531
 265254 240573 238961 201966 202969 206484 204528 202900 155503 897015
 217780 211854 210161 197716 152449 772844 210396 191972]
resultado_julgamento:
  Missing 

In [20]:
## Vamos remover esses valores faltantes da tabela filtrada

dados_limpos_3 = dados_filtrados_3.dropna()

for var in variaveis_interesse_3:
    observacoes_variavel = len(dados_limpos_3[var])
    print(f"{var}: {observacoes_variavel} observações")

ID_processo: 88 observações
resultado_julgamento: 88 observações
filhos: 88 observações


In [ ]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 3: resultado_julgamento vs existência de filhos

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'filhos', dados_limpos_3)
resultados['resultado_julgamento_vs_filhos'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs filhos ===
Tabela de contigência
filhos                0.0  1.0
resultado_julgamento          
0                       4   13
1                      60   11
Odds Ratio: 0.0564
P-value: 0.000003
Significância: *** (p < 0.001)

resultado_julgamento_vs_filhos:
  Odds Ratio: 0.0564
  P-valor: 0.000003
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Filhos**

1. Odds Ratio: 0.056 (< 1). Associação negativa entre as variáveis em questão. Ter filhos diminui as chances de ser condenado. De acordo com a amostra coletada, a chance de absolvição em réus sem filhos é apenas ~5,6% da chance de absolvição de réus com filhos. P-valor: 0.000003, significância a 1% (muito significante).

2.  Dos casos com filho na amostra, 11 foram condenados e 13 absolvidos. 

3.  Dos casos sem filhos na amostra, 60 foram condenados e 4 absolvidos. 

**Conclusão:** *réus com filhos têm muito mais chance de serem absolvidos*.

**• O p-valor está abaixo de 0.05 (0.000003), indicando uma associação forte entre resultado do julgamento e a existência de filhos.**

In [22]:
## Analisando consentimento familiar

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_4 = ['ID_processo','resultado_julgamento', 'consentimento_familiar']
dados_filtrados_4 = dados[variaveis_interesse_4].copy()
dados_filtrados_4 = dados_filtrados_4[dados_filtrados_4['ID_processo'] != 202146] ## removendo um dos processos
print(dados_filtrados_4)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_4)}")

     ID_processo  resultado_julgamento  consentimento_familiar
0         148088                     1                     1.0
1         203354                     1                     0.0
2         211280                     1                     0.0
3         264516                     1                     0.0
4         849912                     1                     0.0
..           ...                   ...                     ...
126       197716                     0                     1.0
127       152449                     0                     0.0
128       772844                     0                     1.0
130       210396                     0                     1.0
131       191972                     0                     1.0

[131 rows x 3 columns]
Total de observações originais: 132
Total de observações final: 131


In [ ]:
## Verificando missing values

for var in variaveis_interesse_4:
    missing_count = dados_filtrados_4[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_4)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_4[var].unique()}")

# 7% da amostra tem missing values. 

ID_processo:
  Missing values: 0 (0.0%)
  Valores únicos: [148088 203354 211280 264516 849912 804741 254352 202422 224010 200221
 184262 132092 835236 173028 191800 193481 694900 184789 174559 146576
 143912 280782 212700 215458 885416 913497 208333 212840 871265 249385
 157443 239068 259005 176898 176979 527774 171531 176559 169452 149635
 175618 140372 143672 155742 158776 231453 134780 195164 831541 800685
 795482 210952 208602 722014 681080 676155 199917 197973 197199 208631
 237957 147619 183064 201216 195723 169551 208489 226153 185259 630372
 188501 189990 142704 163831 153688 379763 477088 382196 387667 160739
 185437 163595 176254 171010 136005 165091 155347 158511 162216 154015
 728575 110419 283470 966382 170792 216534 961744 159217 270098 211850
 214764 231698 248059 279235 166443 172188 118368 221858 230572 201531
 265254 240573 238961 201966 202969 206484 204528 202900 155503 897015
 217780 211854 210161 197716 152449 772844 210396 191972]
resultado_julgamento:
  Missing 

In [ ]:
## Vamos remover esses valores faltantes da tabela filtrada

dados_limpos_4 = dados_filtrados_4.dropna()

for var in variaveis_interesse_4:
    observacoes_variavel = len(dados_limpos_4[var])
    print(f"{var}: {observacoes_variavel} observações")

## Amostra final com 122 observações.

ID_processo: 122 observações
resultado_julgamento: 122 observações
consentimento_familiar: 122 observações


In [25]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 4: resultado_julgamento vs consentimento familiar

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'consentimento_familiar', dados_limpos_4)
resultados['resultado_julgamento_vs_consentimento_familiar'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs consentimento_familiar ===
Tabela de contigência
consentimento_familiar  0.0  1.0
resultado_julgamento            
0                         7   13
1                        83   19
Odds Ratio: 0.1233
P-value: 0.000069
Significância: *** (p < 0.001)

resultado_julgamento_vs_consentimento_familiar:
  Odds Ratio: 0.1233
  P-valor: 0.000069
  Significância: *** (p < 0.001)


## Interpretação

#### **Resultado do Julgamento vs Consentimento Familiar**

1. Odds Ratio: 0.1233 (< 1). Associação negativa entre as variáveis em questão. O consentimento familiar diminui as chances de ser condenado. De acordo com a amostra coletada, a chance de absolvição em casos sem consentimento familiar é ~12,3% da chance de absolvição de casos com consentimento familiar. P-valor: 0.000069, significância a 1% (muito significante).

2.  Dos casos com consentimento familiar na amostra, 19 foram condenados e 13 absolvidos. 

3.  Dos casos sem consentimento familiar amostra, 83 foram condenados e 7 absolvidos. 

**Conclusão:** *quando há consentimento familiar, os réus tem mais chance de serem absolvidos*.

**• O p-valor está abaixo de 0.05 (0.000003), indicando uma associação forte entre resultado do julgamento e o consentimento familiar.**

In [26]:
## Analisando consentimento da vítima

## Filtrando apenas para as variáveis de interesse

variaveis_interesse_5 = ['ID_processo','resultado_julgamento', 'consentimento_vitima']
dados_filtrados_5 = dados[variaveis_interesse_5].copy()
dados_filtrados_5 = dados_filtrados_5[dados_filtrados_5['ID_processo'] != 202146] ## removendo um dos processos
print(dados_filtrados_5)
print(f"Total de observações originais: {len(dados)}")
print(f"Total de observações final: {len(dados_filtrados_5)}")

     ID_processo  resultado_julgamento  consentimento_vitima
0         148088                     1                   1.0
1         203354                     1                   1.0
2         211280                     1                   1.0
3         264516                     1                   1.0
4         849912                     1                   1.0
..           ...                   ...                   ...
126       197716                     0                   1.0
127       152449                     0                   1.0
128       772844                     0                   1.0
130       210396                     0                   1.0
131       191972                     0                   1.0

[131 rows x 3 columns]
Total de observações originais: 132
Total de observações final: 131


In [ ]:
## Verificando missing values

for var in variaveis_interesse_5:
    missing_count = dados_filtrados_5[var].isna().sum()
    missing_percent = (missing_count/len(dados_filtrados_5)) * 100
    print(f"{var}:")
    print(f"  Missing values: {missing_count} ({missing_percent:.1f}%)")
    print(f"  Valores únicos: {dados_filtrados_5[var].unique()}")

# 3% da amostra tem missing values. 

ID_processo:
  Missing values: 0 (0.0%)
  Valores únicos: [148088 203354 211280 264516 849912 804741 254352 202422 224010 200221
 184262 132092 835236 173028 191800 193481 694900 184789 174559 146576
 143912 280782 212700 215458 885416 913497 208333 212840 871265 249385
 157443 239068 259005 176898 176979 527774 171531 176559 169452 149635
 175618 140372 143672 155742 158776 231453 134780 195164 831541 800685
 795482 210952 208602 722014 681080 676155 199917 197973 197199 208631
 237957 147619 183064 201216 195723 169551 208489 226153 185259 630372
 188501 189990 142704 163831 153688 379763 477088 382196 387667 160739
 185437 163595 176254 171010 136005 165091 155347 158511 162216 154015
 728575 110419 283470 966382 170792 216534 961744 159217 270098 211850
 214764 231698 248059 279235 166443 172188 118368 221858 230572 201531
 265254 240573 238961 201966 202969 206484 204528 202900 155503 897015
 217780 211854 210161 197716 152449 772844 210396 191972]
resultado_julgamento:
  Missing 

In [ ]:
## Vamos remover esses valores faltantes da tabela filtrada

dados_limpos_5 = dados_filtrados_5.dropna()

for var in variaveis_interesse_5:
    observacoes_variavel = len(dados_limpos_5[var])
    print(f"{var}: {observacoes_variavel} observações")

## Amostra final com 127 observações.

ID_processo: 127 observações
resultado_julgamento: 127 observações
consentimento_vitima: 127 observações


In [29]:
## Vamos executar o teste

# Inicializar o dicionário de resultados

resultados = {}

## Teste 5: resultado_julgamento vs consentimento da vítima

odds1, p1, sig1 = teste_fisher('resultado_julgamento', 'consentimento_vitima', dados_limpos_5)
resultados['resultado_julgamento_vs_consentimento_vitima'] = {'odds_ratio': odds1, 'p_value': p1, 'significancia': sig1}

for teste, resultado in resultados.items():
    if resultado['odds_ratio'] is not None:
        print(f"\n{teste}:")
        print(f"  Odds Ratio: {resultado['odds_ratio']:.4f}")
        print(f"  P-valor: {resultado['p_value']:.6f}")
        print(f"  Significância: {resultado['significancia']}")
    else:
        print(f"\n{teste}: Teste não executado")


=== Teste de Fisher: resultado_julgamento vs consentimento_vitima ===
Tabela de contigência
consentimento_vitima  0.0  1.0
resultado_julgamento          
0                       0   20
1                      19   88
Odds Ratio: 0.0000
P-value: 0.041795
Significância: * (p < 0.05)

resultado_julgamento_vs_consentimento_vitima:
  Odds Ratio: 0.0000
  P-valor: 0.041795
  Significância: * (p < 0.05)


## Interpretação

#### **Resultado do Julgamento vs Consentimento Familiar**

1. Odds Ratio: 0 (tende a zero ou é indefinido). Há zero na tabela de contingência. A chance de absolvição sem consentimento é praticamente inexistente na amostra.

2.  Dos casos com consentimento da vítima na amostra, 88 foram condenados e 20 absolvidos. 

3.  Dos casos sem consentimento da vítima na amostra, 19 foram condenados e 0 absolvidos. 

**Conclusão:** *quando há consentimento da vítima, os réus tem mais chance de serem absolvidos*.

**• O p-valor está abaixo de 0.05 (0.041795), indicando uma associação não aleatória entre resultado do julgamento e o consentimento da vítima, todavia, essa associação não é tão forte ou robusta como nos casos anteriores - pode ser o caso aqui da dificuldade de documentar o consentimento da vítima, que não é claro nos processos.**